## What is Jupyter notebook?

Direct introduction from [Jupyter notebook website](http://jupyter.org/)




## What is pytraj?

pytraj is a Python package binding to the popular cpptraj program. pytraj is written do extend the flexibility of cpptraj and to expose cpptraj's functionanity to Python's ecossytem, such as numpy, pandas, matplotlib, ...

## What can you learn from this tutorial

- Use Jupyter notebook for interective data exploration

- Use pytraj to perform basis analysis such computing RMSD, center of mass distance, dihedral angles, ...

- Use matplotlib and pandas for analyzing output from pytraj


## Requirement

- AmberTools >= 16
- If you allow AMBER to install Python distribution from Miniconda, you has all needed packages for this tutorial.
- If not, you need to install (either using `pip` or `conda`)
    - matplotlib
    - pandas
    - jupyter notebook

### Start notebook

```bash
    amber.jupyter notebook
```